In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
link_template = 'https://www.myjobmag.co.ke'


def add_records(tags, records):
    added_job = False
    for elem in tags:
        attrs = elem.attrs
        if 'href' not in attrs:
            continue
        href = attrs['href']
        if '/job/' in href:
            records.append({
                'title': elem.text,
                'link': f'{link_template}{href}'
            })
            added_job = True
    return added_job


In [3]:
job_title = 'Business Analyst'

locations = ['Nairobi', 'Bomet']

first_site = 'https://www.myjobmag.co.ke/search/jobs/'
parameters = {
    'q': job_title,
    'location': '--'.join(locations)
}

records = []

added_job = True
current_page = 1
start_time = time.time()
while added_job:
    current_time = time.time()
    print(f'Current page: {current_page}, Spent time: {int((current_time - start_time) // 60)} min', end='\r')
    page = requests.get(
        url=first_site,
        params=parameters
    )

    soup = BeautifulSoup(page.content, 'html.parser')
    added_job = add_records(
        tags=soup.find_all('a', recursive=True),
        records=records
    )
    current_page += 1
    parameters['currentpage'] = current_page
jobs_table = pd.DataFrame.from_records(records)
jobs_table.to_csv('myjobmag_jobs.csv')
